In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
import GCRCatalogs
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm

Check what version we're using

In [ ]:
clmm.__version__

### 1. Prepare a CLMM GalaxyCluster object from cosmoDC2 

#### Read in the extragalactic catalog cosmoDC2

In [ ]:
extragalactic_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

In [ ]:
cosmo = extragalactic_cat.cosmology
cosmo

#### Get the list of halos with `M > mmin` in the redshift range `[zmin, zmax]`

In [ ]:
# get list of massive halos in a given redshift and mass range
mmin = 5.e14
zmin = 0.3
zmax = 0.4

massive_halos = extragalactic_cat.get_quantities(['halo_mass','hostHaloMass','redshift','ra', 'dec'],
                                                 filters=[f'halo_mass > {mmin}','is_central==True',
                                                          f'redshift>{zmin}', f'redshift<{zmax}'])
N_cl = len(massive_halos['halo_mass'])
print(f'There are {N_cl} clusters available')

#### Select the most massive one

In [ ]:
# Selecting the most massive one

select = massive_halos['halo_mass'] == np.max(massive_halos['halo_mass'])

ra_cl = massive_halos['ra'][select][0]
dec_cl = massive_halos['dec'][select][0]
z_cl = massive_halos['redshift'][select][0]
mass_cl =massive_halos['halo_mass'][select][0]
print (f'The most massive cluster is in ra = {ra_cl:.2f} deg, dec = {dec_cl:.2f} deg, z = {z_cl:.2f}, with mass = {mass_cl:.2e} Msun')

#### Apply coordinates, redshift and magnitude cuts to select backgroud galaxies around the cluster

- Box of 0.6 deg around the cluster center
- Galaxies with z > z_clsuter + 0.1
- Galaxies with mag_i < 25 

Here, we're directly gathering the shear components $\gamma_{1,2}$ and the convergence $\kappa$ from the cosmoDC2 catalog. See the `DC2_gt_profiles` notebook to see how to also use the intrinsic ellipticities of the galaxies to compute observed ellipticities including intrinsic and shear components.

In [ ]:
ra_min, ra_max = ra_cl-0.3, ra_cl+0.3
dec_min, dec_max = dec_cl-0.3, dec_cl+0.3
z_min = z_cl + 0.1
mag_i_max = 25

coord_filters = ['ra >= {}'.format(ra_min),'ra < {}'.format(ra_max),'dec >= {}'.format(dec_min),'dec < {}'.format(dec_max)]
z_filters = ['redshift >= {}'.format(z_min)]
mag_filters = ['mag_i < {}'.format(mag_i_max)]
gal_cat = extragalactic_cat.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2','redshift', 'convergence'],\
                                           filters=(coord_filters + z_filters + mag_filters))



#### To compute a reduced tangential shear profile using CLMM, we first need to transform the shear into ellipticities. 
- The CLMM function `convert_shapes_to_epsilon` convert any shape measurements into the corresponding ellipticities.
- Then, we build the astropy table of the galaxy catalog that will be used to instantiate a CLMM GalaxyCluster object.

In [ ]:
e1, e2 = clmm.utils.convert_shapes_to_epsilon(gal_cat['shear_1'],gal_cat['shear_2'],\
                                              shape_definition='shear',kappa=gal_cat['convergence'])

#store the results into an astropy table
dat = Table([gal_cat['ra'],gal_cat['dec'],e1, e2,gal_cat['redshift'],gal_cat['galaxy_id']], 
            names=('ra','dec', 'e1', 'e2', 'z','id'))

cl = clmm.GalaxyCluster('CL', ra_cl, dec_cl, z_cl, dat)   

In [ ]:
# Quick check of the redshift distribution of the galaxies in the catalog

print(f'Number of galaxies in the catalog: Ngal = {len(cl.galcat)}')
tmp = plt.hist(cl.galcat['z'], bins=30)
plt.xlabel('z')
plt.ylabel('Count')

### 2. Use CLMM to compute the reduced tangential shear profile

#### Compute the tangential and cross shear profiles

In [ ]:
bin_edges = clmm.polaraveraging.make_bins(0.15, 10, 15, method='evenlog10width') # in Mpc
cl.compute_shear(geometry="flat")
cl.make_shear_profile("radians", "Mpc", bins=bin_edges,cosmo=cosmo)

### 3. Sanity check: use CLMM to compute the corresponding NFW model, given the mass the halo

- The mass definition used in cosmoDC2 is the friend-of-friend mass with linking length b=0.168. In CLMM, the default mass definition is $M_{200,m}$: it uses an overdensity parameter $\Delta=200$ with respect to the matter density. Here, we are directly using $M_{\rm fof}$ in the modeling functions of CLMM, which is inconsistent. However, the goal here is to check that model and data are roughly in agreement. 
- The model should take into account the redshift distribution of the background galaxies. Here, we simply use the average redshift of the galaxy catalog as this is a quick sanity check that things behave as expected. 

In [ ]:
reduced_shear = clmm.predict_reduced_tangential_shear(cl.profile['radius']*cosmo.h, mass_cl*cosmo.h, 4.,z_cl, cl.profile['z'], cosmo,
                                                      delta_mdef=200,halo_profile_model='nfw')  

In [ ]:
plt.errorbar(profile['radius'],profile['gt'],yerr=profile['gt_err'])
plt.plot(profile['radius'],reduced_shear_model, label='model shear')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.xlabel('R (Mpc)')
plt.ylabel('<shear>')

Data and model are in rough agreement at large radii. In the inner region, the lack of resolution of the DC2 simulations yield an unphysical attenuation of the signal. This was remarked upon in the cosmoDC2 paper (https://arxiv.org/abs/1907.06530).